In [1]:
import os
import cv2
from vidaug import augmentors as va

In [2]:
import numpy as np

# restore the old alias so vidaug’s code will keep working
if not hasattr(np, "float"):
    np.float = float

In [ ]:
# 50%‐chance wrapper
sometimes = lambda aug: va.Sometimes(0.5, aug)

seq = va.Sequential([

    sometimes( va.Salt()),
    sometimes( va.Pepper()),

    # tiny shifts/scales
    sometimes( va.RandomTranslate(3, 3) ),  # ±10% shift

    # brightness / contrast jitter
    sometimes( va.Add(20) ),          # ±20% brightness
    sometimes( va.Add(-20) ),          # ±20% brightness
    sometimes( va.Multiply(1.2) ),            # contrast factor 0.8–1.2

    # low‑level blur
    sometimes( va.GaussianBlur(0.5) ),        # small blur

    sometimes( va.HorizontalFlip() ),
    
    sometimes( va.ElasticTransformation(alpha=5, sigma=1) ),
])

In [ ]:


# where your original videos live
input_root  = "dataset/daisee-separated/Train"
# where to dump augmented videos
output_root = "dataset/daisee-separated/Train"

# make sure vidaug uses numpy arrays in shape (N_frames, H, W, C)
for cls in ["0"]:
    in_dir  = os.path.join(input_root,  cls)
    out_dir = os.path.join(output_root, cls+"/Augmented")
    os.makedirs(out_dir, exist_ok=True)

    for fname in os.listdir(in_dir):
        if not fname.lower().endswith((".mp4", ".avi")):
            continue

        video_path = os.path.join(in_dir, fname)
        cap = cv2.VideoCapture(video_path)
        fps    = cap.get(cv2.CAP_PROP_FPS)
        w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frames = []

        # read all frames
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # convert BGR to RGB for vidaug
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        cap.release()

        # now produce 15 augmented variants
        base, _ = os.path.splitext(fname)
        fourcc  = cv2.VideoWriter_fourcc(*"XVID")
        for aug_i in range(1, 16):
            aug_frames = seq(frames)

            out_path = os.path.join(out_dir, f"{base}_aug{aug_i}.avi")
            writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

            # vidaug returns RGB; convert back to BGR for cv2
            for f in aug_frames:
                # ensure uint8 depth:
                if f.dtype != np.uint8:
                    # if floats 0–1, scale; if floats already 0–255, just clip+cast:
                    f = np.clip(f, 0, 255).astype(np.uint8)
        
                # now convert and write
                bgr = cv2.cvtColor(f, cv2.COLOR_RGB2BGR)
                writer.write(bgr)
            writer.release()

In [ ]:
# make sure vidaug uses numpy arrays in shape (N_frames, H, W, C)
for cls in ["1"]:
    in_dir  = os.path.join(input_root,  cls)
    out_dir = os.path.join(output_root, cls+"/Augmented")
    os.makedirs(out_dir, exist_ok=True)

    for fname in os.listdir(in_dir):
        if not fname.lower().endswith((".mp4", ".avi")):
            continue

        video_path = os.path.join(in_dir, fname)
        cap = cv2.VideoCapture(video_path)
        fps    = cap.get(cv2.CAP_PROP_FPS)
        w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frames = []

        # read all frames
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # convert BGR to RGB for vidaug
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        cap.release()

        # now produce 3 augmented variants
        base, _ = os.path.splitext(fname)
        fourcc  = cv2.VideoWriter_fourcc(*"XVID")
        for aug_i in range(1, 4):
            aug_frames = seq(frames)

            out_path = os.path.join(out_dir, f"{base}_aug{aug_i}.avi")
            writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

            # vidaug returns RGB; convert back to BGR for cv2
            for f in aug_frames:
                # ensure uint8 depth:
                if f.dtype != np.uint8:
                    # if floats 0–1, scale; if floats already 0–255, just clip+cast:
                    f = np.clip(f, 0, 255).astype(np.uint8)
        
                # now convert and write
                bgr = cv2.cvtColor(f, cv2.COLOR_RGB2BGR)
                writer.write(bgr)
            writer.release()